In [1]:
import numpy as np

import pyoti.sparse as oti

# Set pyoti to print all coefficients.
oti.set_printoptions(terms_print=-1)
np.set_printoptions(linewidth=120)

# tseuqlib
import tseuqlib as tuq
import tseuqlib.rv_moments as rv
import tseuqlib.oti_moments as moti
import tseuqlib.oti_util as uoti

/opt/anaconda3/envs/pyoti/lib/python3.9/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


Let's define the function here
$$
y=f(\mathbf{x}) = (1 + 2x_{1} + 3x_{2} +5x_{3} + 7x_{4})^3
$$

In [2]:
def funct(x):
    """
    Equation 2.92 in 
    
    INPUTS:
    - xi: OTI number.
    """
    
    return (1 + 2*x[0] + 3*x[1] + 5*x[2] + 7*x[3])**3

Define the random variables, $x_{1}, x_{2}, x_{3}, x_{4}$ as Kumaraswamy distributions

In [3]:
rv_pdf_name = np.array(['K', 'K', 'K', 'K']) # Assigning Kumaraswamy distribution to the random variables

shape = [1, 1, 2, 2]
rv_a = np.array([shape[0], 
                 shape[1],
                 shape[2],
                 shape[3]]) 

scale = [3, 8, 6, 4]
rv_b = np.array([scale[0],
                 scale[1],
                 scale[2],
                 scale[3]])

# Number of dimensions
n_dim = len(shape)

Now that we have our random variables defined with the type of distribution and shape, scale parameters,

We can compute the expected value and standard deviation of each random variable using ``rv_mean_from_ab()`` function.

In [4]:
rv_params = rv.rv_mean_from_ab([rv_pdf_name, rv_a, rv_b])

print('-'*40)
print('|E[xi]       |STD[xi]     |Var[xi]     |')
print('-'*40)
for i in range(n_dim):
    mean_xi = rv_params[3][i]
    std_xi = rv_params[4][i]
    var_xi = std_xi**2
    print(f'|{mean_xi:.5f}     |{std_xi:.5f}     |{var_xi:.5f}     |')

----------------------------------------
|E[xi]       |STD[xi]     |Var[xi]     |
----------------------------------------
|0.25000     |0.19365     |0.03750     |
|0.11111     |0.09938     |0.00988     |
|0.34099     |0.16304     |0.02658     |
|0.40635     |0.18676     |0.03488     |


Now we construct our input variables by first adding an imaginary base to the expected value of each random variable. Let's use a 3rd order expansion

In [5]:
tse_order = 3

x = [0]*n_dim
for d in range(n_dim):
    x[d] = rv_params[3][d] + oti.e(int(d+1), order=tse_order)
print(x)

[0.25 + 1 * e([1]), 0.111111 + 1 * e([2]), 0.340992 + 1 * e([3]), 0.406349 + 1 * e([4])]


Now we simply pass ``x`` into our function.

In [6]:
y = funct(x)
print(y)

260.029 + 244.436 * e([1]) + 366.654 * e([2]) + 611.09 * e([3]) + 855.527 * e([4]) + 76.5929 * e([[1,2]]) + 229.779 * e([1,2]) + 172.334 * e([[2,2]]) + 382.964 * e([1,3]) + 574.447 * e([2,3]) + 478.705 * e([[3,2]]) + 536.15 * e([1,4]) + 804.225 * e([2,4]) + 1340.38 * e([3,4]) + 938.263 * e([[4,2]]) + 8 * e([[1,3]]) + 36 * e([[1,2],2]) + 54 * e([1,[2,2]]) + 27 * e([[2,3]]) + 60 * e([[1,2],3]) + 180 * e([1,2,3]) + 135 * e([[2,2],3]) + 150 * e([1,[3,2]]) + 225 * e([2,[3,2]]) + 125 * e([[3,3]]) + 84 * e([[1,2],4]) + 252 * e([1,2,4]) + 189 * e([[2,2],4]) + 420 * e([1,3,4]) + 630 * e([2,3,4]) + 525 * e([[3,2],4]) + 294 * e([1,[4,2]]) + 441 * e([2,[4,2]]) + 735 * e([3,[4,2]]) + 343 * e([[4,3]])


As a result, we obtain a 3rd-order Taylor series expansion of $f(x)$. The imaginary bases are present for each perturbed random variable.

Now, let's compute the central moments of $f(x)$.

In [7]:
# Define the order of moments
rv_moments_order = int((tse_order*4))

rv_moments = rv.rv_central_moments(rv_params, rv_moments_order)

# Compute central moments of the random variables
rv_moments.compute_central_moments()

# Print out central moments
print('-'*58)
print('|mu_i     |x1          |x2          |x3          |x4     |')
print('-'*58)
for i in range(len(rv_moments.rv_mu)):
    mu_i = rv_moments.rv_mu[i]
    print(f'|mu_{i+1}     |{mu_i[0]:.5f}     |{mu_i[1]:.5f}     |{mu_i[2]:.5f}     |{mu_i[3]:.5f}|')

----------------------------------------------------------
|mu_i     |x1          |x2          |x3          |x4     |
----------------------------------------------------------
|mu_1     |0.00000     |0.00000     |0.00000     |0.00000|
|mu_2     |0.03750     |0.00988     |0.02658     |0.03488|
|mu_3     |0.00625     |0.00140     |0.00136     |0.00121|
|mu_4     |0.00435     |0.00052     |0.00180     |0.00289|
|mu_5     |0.00167     |0.00017     |0.00026     |0.00029|
|mu_6     |0.00092     |0.00006     |0.00019     |0.00034|
|mu_7     |0.00046     |0.00003     |0.00005     |0.00006|
|mu_8     |0.00026     |0.00001     |0.00003     |0.00005|
|mu_9     |0.00014     |0.00001     |0.00001     |0.00001|
|mu_10     |0.00008     |0.00000     |0.00000     |0.00001|
|mu_11     |0.00005     |0.00000     |0.00000     |0.00000|
|mu_12     |0.00003     |0.00000     |0.00000     |0.00000|


Now from the random variable moments ``rv_moments``, we need to construct the joint probability distribution function from the random variables.

In [8]:
# Build the joint distribution from uncorrelated variables.
rv_mu_joint = uoti.build_rv_joint_moments(rv_moments.rv_mu)

# Create oti_moments object
oti_mu = moti.tse_uq(n_dim, rv_mu_joint)

Now we can compute and print out the moments of $f(x)$

In [9]:
# Expectation
mu_y   = oti_mu.expectation(y)
print('mu_y =', mu_y)

# variance
mu2     = oti_mu.central_moment(y, 2)
print('mu2 =', mu2)

# Third central moment
mu3 = oti_mu.central_moment(y, 3)
print('mu3 =', mu3)

# Fourth central moment
mu4 = oti_mu.central_moment(y, 4)
print('mu4 =', mu4)

mu_y = 310.7254316054317
mu2 = 51653.02542169817
mu3 = 16706254.302919738
mu4 = 15315376784.846674


Let's see how accurate the expected value and central moments of $f(x)$ against analytical solutions of metrics, see below.

We will use relative error calculations.

In [10]:
# Analytical solutions of metrics (precomputed for the example in Section 3.1)
ev_an  = 209949406/675675
mu2_an = 400885535046969013/7761123995625
mu3_an = 16706254.3029196
mu4_an = 15315376784.8466

In [11]:
def relative_error(y, y_hat):
    return np.abs(y-y_hat)/np.abs(y)

In [12]:
print(relative_error(ev_an, mu_y))
print(relative_error(mu2_an, mu2))
print(relative_error(mu3_an, mu3))
print(relative_error(mu4_an, mu4))

3.6587554850025586e-16
1.9720704792594363e-15
8.250547282702534e-15
4.856987700967787e-15


What we see is that the expected value and central moments using TSEUQlib yield machine accurate results.